In [0]:

#%pip install megadetector
#%pip install dbutils
#%pip install yolov5
#%pip install torch 
#%pip install torchvision
# %pip install numpy==1.26.4
# %pip install supervision
# %pip install lightning
# %pip install omegaconf
#%pip install ultralytics
%pip install torch
%pip install ultralytics
%pip install pytesseract
%pip install config
%pip install yolov5
%pip install faiss-cpu
#%pip install faiss-cpu

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Using cached faiss_cpu-1.13.1-cp310-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (7.6 kB)
Using cached faiss_cpu-1.13.1-cp310-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (23.7 MB)
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
#always needed
import config
import os
import pandas as pd
import torch #GPU version
import numpy as np
import io

#data preprocessing librariesa
import cv2,pytesseract
from PIL import Image,ImageStat
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import functions as when
from pyspark.sql import Row

from pyspark.sql.functions import col, input_file_name, lit, current_timestamp, split, element_at, regexp_extract
from pyspark.sql.functions import col, substring, length
from pyspark.sql.types import StructType, StructField, IntegerType,BooleanType, FloatType, StringType,BinaryType,LongType, TimestampType
from pyspark.sql.functions import sha2, expr
from pyspark.sql.functions import hash as spark_hash, pmod
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from delta.tables import DeltaTable

#model related libraries
from ultralytics import YOLO #for YOLOv9
from pathlib import Path

#others
import requests # Define the URL and the local file path
import json
from datetime import datetime
from ultralytics import YOLO

In [0]:
# define the year of photoes for table names
photo_year="2024"

# schema of the output
schema= "meadowbank_prod"

# define output of this model's name and location, ex. md24v6 then final table will be 
process_name="md24_tws"

# Volume root path
volume_root = "/Volumes/meadowbank_prod/remote_cameras/mbk_remote_cameras_blob/2024/"

# designated folder for animal output images
output_uc_folder="/Volumes/meadowbank_prod/remote_cameras/mbk_remote_cameras_blob/results/2024_TWS/wildlife_test/"

# generate the Volume table in catalog with 2.5 million images including final model output
manifest_table=f"{schema}.remote_cameras.{process_name}_manifest_{photo_year}"

manifest_timestamp_table=f"{schema}.remote_cameras.{process_name}_manifest_timestamp_{photo_year}"

manifest_prefiltered_table = f"{schema}.remote_cameras.{process_name}_manifest_prefiltered_{photo_year}"

model_table=f"{schema}.remote_cameras.{process_name}_megadetector_model_{photo_year}"

model_output_table=f"{schema}.remote_cameras.{process_name}_megadetector_v5a_v6byolo9c_results_{photo_year}"

manifest_table_final=f"{schema}.remote_cameras.{process_name}_manifest_{photo_year}_final"

# manifest final output table for final_category to classify images
animal_confidence_threshold=0.5
vehicle_confidence_threshold=0.3
human_confidence_threshold=0.3

In [0]:
#load the manifest table meadowbank_prod.remote_cameras.md24_tws_manifest
df = spark.sql(f"select * from {manifest_table}")

#load the manifest table with timestamp meadowbank_prod.remote_cameras.md24_tws_manifest_timestamp 
df_ts = spark.sql(f"select * from {manifest_timestamp_table}")

#load the manifest table
# df = spark.sql(f"select * from meadowbank_prod.remote_cameras.md24_tws_manifest")

#load the manifest table with timestamp
# df_ts = spark.sql(f"select * from meadowbank_prod.remote_cameras.md24_tws_manifest_timestamp")

In [0]:
# create tm_indicator
df_ts = df_ts.withColumn(
    "TM_flag",
    F.when(F.col("timestamp").endswith("0:00"), "T").otherwise("M")
)

df_ts = df_ts.withColumnRenamed("path", "image_path")

df_join=df.join(df_ts, on='image_path')

# add a column 'pre-filtered' with logic saying if file_size<105 or tm_indicator=='T' then 'True' else 'False'
df_pre_filtered = df_join.withColumn(
    "pre_filtered",
    F.when(
        (col("file_size_kb") < 105) | (col("TM_flag") == "T"),
        True
    ).otherwise(False)
)

In [0]:
# Write to Catalog Volum, if already did, no need to redo again
# df_pre_filtered.write.format("delta") \
#     .mode("overwrite") \
#     .option("overwriteSchema", "true") \
#     .saveAsTable(manifest_prefiltered_table)

# print("✅ joined_df with pre_filtered column successfully written to {manifest_prefiltered_table}")

In [0]:
df_pre_filtered.groupBy("pre_filtered") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+------------+-------+
|pre_filtered|  count|
+------------+-------+
|       false|2173531|
|        true| 326800|
+------------+-------+



In [0]:
# for watermark analyses -> B10
# identify images will be excluded by the filter
a = df_pre_filtered.filter(col("pre_filtered") == True)
a.count() 

# identify images will NOT be excluded by the filter
filtered_df = df_pre_filtered.filter(col("pre_filtered") == False)
filtered_df.count()

2173531

In [0]:
#load the table will run through the models
df_model = spark.sql(f"select * from {model_table}")
df_model.count()

1279887

In [0]:
#get all the model image path
model_paths = df_model.select("image_path").distinct()

#create a new column mg5a_binary to df_pre_filtered table
df_pre_filtered_flagged = (
    df_pre_filtered
        .join(model_paths.withColumn("in_model", F.lit(True)),
              on="image_path",
              how="left")
        .withColumn(
            "mg5a_binary",
            F.when(
                (F.col("pre_filtered") == True) & (F.col("in_model") == True),
                True
            )
            .otherwise(False)
        )
        .drop("in_model")
)


In [0]:
# for waterfall analyses true count -> B13
df_pre_filtered_flagged.groupBy("mg5a_binary") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+-----------+-------+
|mg5a_binary|  count|
+-----------+-------+
|      false|2409233|
|       true|  91098|
+-----------+-------+



In [0]:
#df_pre_filtered_flagged.show(5,truncate=False)

In [0]:
df_model_output=spark.sql(f"select * from {model_output_table}")
df_model_output.count()

956874

In [0]:
#df_model_output.show(5,truncate=False)

In [0]:
#Remove "dbfs:" from image_path
df_pre_clean = df_pre_filtered_flagged.withColumn(
    "image_path",
    F.regexp_replace("image_path", "^dbfs:", "")
)


In [0]:
#df_model_output.show(5,truncate=False)

In [0]:
import pyspark.sql.functions as F

# exclude all tagged images from camera 4
df_model_output_no_camera4 = df_model_output.filter(~F.col("image_path").contains("Camera 4"))

#apply threshold to individual images
#For vehicles (category 3) → conf_md5a ≥ vehicle_threshold AND conf_md9c ≥ vehicle_threshold
#For animals (category 1) → conf_md5a ≥ animal_threshold AND conf_md9c ≥ animal_threshold
#For humans (category 2) → conf_md5a ≥ human_threshold AND conf_md9c ≥ human_threshold

df_results = (
    df_model_output_no_camera4
    .filter(
        (
            (F.col("category") == "3") &
            (F.col("conf_md5a") >= vehicle_confidence_threshold) &
            (F.col("conf_md9c") >= vehicle_confidence_threshold)
        ) |
        (
            (F.col("category") == "1") &
            (F.col("conf_md5a") >= animal_confidence_threshold) &
            (F.col("conf_md9c") >= animal_confidence_threshold)
        ) |
        (
            (F.col("category") == "2") &
            (F.col("conf_md5a") >= human_confidence_threshold) &
            (F.col("conf_md9c") >= human_confidence_threshold)
        )
    )
)


In [0]:
from pyspark.sql import functions as F

#the numbers of distinct images for each category, used for output workflow
df_results.groupBy("category") \
    .agg(F.countDistinct("image_path").alias("distinct_image_count")) \
    .orderBy("category") \
    .show()

+--------+--------------------+
|category|distinct_image_count|
+--------+--------------------+
|       1|                5458|
|       2|                1368|
|       3|              334250|
+--------+--------------------+



In [0]:
from pyspark.sql import functions as F

#create mapping from code into category words
category_map = {
    "1": "animals",
    "2": "humans",
    "3": "vehicles"
}

# Convert category into its name
df_named = df_results.withColumn(
    "category_name",
    F.when(F.col("category") == "1", "Animals")
     .when(F.col("category") == "2", "Humans")
     .when(F.col("category") == "3", "Vehicles")
)

df_grouped = (
    df_named
        .groupBy("image_path")
        .agg(
            F.concat_ws(", ", F.array_distinct(F.collect_list("category_name")))
             .alias("final_category")
        )
)


In [0]:
# df_grouped.show(5,truncate=False)

In [0]:
from pyspark.sql import functions as F

df_final_counts = (
    df_grouped
        .groupBy("final_category")
        .agg(F.count("*").alias("image_count"))
        .orderBy("image_count", ascending=False)
)

df_final_counts.show(truncate=False)


+-------------------------+-----------+
|final_category           |image_count|
+-------------------------+-----------+
|Vehicles                 |331929     |
|Animals                  |3516       |
|Animals, Vehicles        |1939       |
|Humans                   |985        |
|Humans, Vehicles         |380        |
|Animals, Humans, Vehicles|2          |
|Animals, Humans          |1          |
+-------------------------+-----------+



In [0]:
#check no duplicates for image_path
distinct_count = df_grouped.select(F.countDistinct("image_path")).first()[0]
print("Distinct image_path:", distinct_count)

Distinct image_path: 338752


In [0]:
from pyspark.sql import functions as F

# 1. Map categories to readable names
df_named = df_model_output.withColumn(
    "category_name",
    F.when(F.col("category") == "1", "animals")
     .when(F.col("category") == "2", "humans")
     .when(F.col("category") == "3", "vehicles")
)

# 2. Create one detection JSON per row 
df_struct = df_named.withColumn(
    "detection_json",
    F.to_json(
        F.struct(
            F.col("category_name").alias("category"),
            F.col("conf_md5a"),
            F.col("bbox_md5a"),
            F.col("conf_md9c"),
            F.col("bbox_md9c")
        )
    )
)

# 3. Group by image_path → collect ALL detections for that image
df_grouped_per_image = (
    df_struct
        .groupBy("image_path")
        .agg(
            F.collect_list("detection_json").alias("json_output")
        )
)


In [0]:
#show the images that belongs to more than one cateogries
# df_grouped.filter(
#     F.size(F.split(F.col("final_category"), ",\\s*")) > 1
# ).show(truncate=False)

In [0]:
#take one of the image_path from output above, QA on output
#for example this file path below has both animals and vehicles, then json_output will show: [{"category":"animals","conf_md5a":0.561,"bbox_md5a":[0.66,0.661,0.018,0.057],"conf_md9c":0.593,"bbox_md9c":[0.66,0.66,0.019,0.057]}, {"category":"vehicles","conf_md5a":0.776,"bbox_md5a":[0.917,0.59,0.167,0.316],"conf_md9c":0.607,"bbox_md9c":[0.917,0.597,0.167,0.299]}]

df_grouped_per_image.filter(
    F.col("image_path") == "/Volumes/meadowbank_prod/remote_cameras/mbk_remote_cameras_blob/2024/FirstPart/Camera 1/2024-05-29/100RECNX/RCNX8065.JPG"
).show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|image_path                                                                                                              |json_output                                                                                                                                                                                                                                                                  |
+------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------

In [0]:
#Perform the join with df_model_output
from pyspark.sql import functions as F

# Step 1 — select only needed columns from df_grouped
df_final_cat = df_grouped.select("image_path", "final_category")

# Step 2 — join df_pre_clean with df_grouped on image_path
df_join1 = (
    df_pre_clean.alias("a")
        .join(df_final_cat.alias("b"), on="image_path", how="left")
)

# Step 3 — select only needed columns from df_grouped_per_image
df_json = df_grouped_per_image.select("image_path", "json_output")

# Step 4 — join the result with df_grouped_per_image
df_manifest_final = (
    df_join1.alias("c")
        .join(df_json.alias("d"), on="image_path", how="left")
)

In [0]:
#QA check, should be the same count to indicate the final_manifest table has all the images populated
df_ts.count()==df_manifest_final.count()

True

In [0]:
#Write to Catalog Volum, if already did, no need to redo again

df_manifest_final.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(manifest_table_final)